In [25]:
import numpy as np
np.random.seed(1337)

from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import SimpleRNN, Activation, Dense
from keras.optimizers import Adam

In [26]:
TIME_STEPS = 28
INPUT_SIZE = 28
BATCH_SIZE = 50
BATCH_INDEX= 0
OUTPUT_SIZE= 10
CELL_SIZE  = 50
LR = 0.001

In [27]:
(X_train, y_train),(X_test,y_test) = mnist.load_data()

In [28]:
X_train = X_train.reshape(-1,28,28)/255
X_test = X_test.reshape(-1,28,28)/255
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)

In [29]:
model = Sequential()
model.add(SimpleRNN(
    batch_input_shape =(None,TIME_STEPS,INPUT_SIZE), 
    output_dim=CELL_SIZE,
    unroll=True   
))

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(batch_input_shape=(None, 28,..., unroll=True, units=50)`
  """


In [30]:
model.add(Dense(OUTPUT_SIZE))
model.add(Activation('softmax'))

In [31]:
adam = Adam(LR)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [34]:
for step in range(4001):
    # data shape = (batch_num, steps, inputs/outputs)
    X_batch = X_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :, :]
    Y_batch = y_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :]
    
    cost = model.train_on_batch(X_batch, Y_batch)
    BATCH_INDEX += BATCH_SIZE
    BATCH_INDEX = 0 if BATCH_INDEX >= X_train.shape[0] else BATCH_INDEX

    if step % 500 == 0:
        cost, accuracy = model.evaluate(X_test, y_test, batch_size=y_test.shape[0], verbose=False)
        print('test cost: ', cost, 'test accuracy: ', accuracy)

test cost:  2.40978717804 test accuracy:  0.0423999987543
test cost:  0.679556488991 test accuracy:  0.792599976063
test cost:  0.443426132202 test accuracy:  0.870999991894
test cost:  0.338477551937 test accuracy:  0.9049000144
test cost:  0.314522087574 test accuracy:  0.90649998188
test cost:  0.288170278072 test accuracy:  0.915499985218
test cost:  0.24478700757 test accuracy:  0.929000020027
test cost:  0.256168693304 test accuracy:  0.922399997711
test cost:  0.267859995365 test accuracy:  0.919900000095


In [35]:
import numpy as np
np.random.seed(1337)  # for reproducibility
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense
from keras.optimizers import Adam

BATCH_START = 0
TIME_STEPS = 20
BATCH_SIZE = 50
INPUT_SIZE = 1
OUTPUT_SIZE = 1
CELL_SIZE = 20
LR = 0.006


def get_batch():
    global BATCH_START, TIME_STEPS
    # xs shape (50batch, 20steps)
    xs = np.arange(BATCH_START, BATCH_START+TIME_STEPS*BATCH_SIZE).reshape((BATCH_SIZE, TIME_STEPS)) / (10*np.pi)
    seq = np.sin(xs)
    res = np.cos(xs)
    BATCH_START += TIME_STEPS
    # plt.plot(xs[0, :], res[0, :], 'r', xs[0, :], seq[0, :], 'b--')
    # plt.show()
    return [seq[:, :, np.newaxis], res[:, :, np.newaxis], xs]

model = Sequential()
# build a LSTM RNN
model.add(LSTM(
    batch_input_shape=(BATCH_SIZE, TIME_STEPS, INPUT_SIZE),       # Or: input_dim=INPUT_SIZE, input_length=TIME_STEPS,
    output_dim=CELL_SIZE,
    return_sequences=True,      # True: output at all steps. False: output as last step.
    stateful=True,              # True: the final state of batch1 is feed into the initial state of batch2
))
# add output layer
model.add(TimeDistributed(Dense(OUTPUT_SIZE)))
adam = Adam(LR)
model.compile(optimizer=adam,
              loss='mse',)

print('Training ------------')
for step in range(501):
    # data shape = (batch_num, steps, inputs/outputs)
    X_batch, Y_batch, xs = get_batch()
    cost = model.train_on_batch(X_batch, Y_batch)
    pred = model.predict(X_batch, BATCH_SIZE)
    plt.plot(xs[0, :], Y_batch[0].flatten(), 'r', xs[0, :], pred.flatten()[:TIME_STEPS], 'b--')
    plt.ylim((-1.2, 1.2))
    plt.draw()
    plt.pause(0.1)
    if step % 10 == 0:
        print('train cost: ', cost)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(batch_input_shape=(50, 20, 1..., return_sequences=True, stateful=True, units=20)`


Training ------------
train cost:  0.509406
train cost:  0.361699
train cost:  0.0589038
train cost:  0.119903
train cost:  0.0761529
train cost:  0.0718034
train cost:  0.0841581
train cost:  0.0810129
train cost:  0.0625276
train cost:  0.0588824
train cost:  0.0512526
train cost:  0.0572694
train cost:  0.0412167
train cost:  0.0368624
train cost:  0.0289653
train cost:  0.0692121
train cost:  0.078529
train cost:  0.13368
train cost:  0.0965155
train cost:  0.132222
train cost:  0.0639728
train cost:  0.0319572
train cost:  0.0307729
train cost:  0.024757
train cost:  0.0349247
train cost:  0.029385
train cost:  0.0352933
train cost:  0.0241589
train cost:  0.0444572
train cost:  0.0566473
train cost:  0.0404938
train cost:  0.0342155
train cost:  0.0233979
train cost:  0.0150866
train cost:  0.0382677
train cost:  0.0470053
train cost:  0.0462276
train cost:  0.0190048
train cost:  0.0250286
train cost:  0.0214047
train cost:  0.0158573
train cost:  0.033296
train cost:  0.0721329